In [1]:
import pandas as pd
from datetime import datetime, timedelta

def generate_cashflow(face_value, coupon_rate, issue_date, maturity_date, frequency=1):
    """Generates a cashflow table for the bond."""
    cashflows = []
    current_date = datetime.strptime(issue_date, "%Y-%m-%d")
    maturity_date = datetime.strptime(maturity_date, "%Y-%m-%d")
    
    while current_date <= maturity_date:
        days_in_period = 366 if ((current_date.year % 4 == 0 and current_date.year % 100 != 0) or (current_date.year % 400 == 0)) else 365
        interest_payment = (coupon_rate * face_value * days_in_period) / 365  # Adjusting for leap years
        
        cashflows.append({
            "Date": current_date.strftime("%Y-%m-%d"),
            "Cashflow": interest_payment
        })
        
        current_date = datetime(current_date.year + 1, current_date.month, current_date.day)
    
    # Add face value repayment at maturity
    cashflows[-1]["Cashflow"] += face_value
    
    df = pd.DataFrame(cashflows)
    print("Generated Cashflows:")
    print(df)
    return df

def calculate_buy_price(cashflows, buy_date, coupon_rate, face_value):
    """Calculates the buy price using present value of future cashflows."""
    buy_date = datetime.strptime(buy_date, "%Y-%m-%d")
    discount_rate = coupon_rate - 0.002  # Using coupon rate - 0.2% as discount rate
    total_present_value = 0
    
    for _, row in cashflows.iterrows():
        cashflow_date = datetime.strptime(row["Date"], "%Y-%m-%d")
        if cashflow_date > buy_date:
            time_diff = (cashflow_date - buy_date).days / 365
            discount_factor = (1 + discount_rate) ** time_diff
            present_value = row["Cashflow"] / discount_factor
            total_present_value += present_value
            print(f"Discounting {row['Cashflow']} on {cashflow_date.strftime('%Y-%m-%d')} with factor {discount_factor:.6f}: {present_value:.2f}")
    
    print(f"Total Present Value of Cashflows: {total_present_value:.2f}")
    buy_price = total_present_value 
    print(f"Computed Buy Price: {buy_price:.2f}")
    return buy_price

def calculate_sell_price(buy_price, sell_date, buy_date, guaranteed_rate, cashflows):
    """Calculates the sell price based on investment period."""
    sell_date = datetime.strptime(sell_date, "%Y-%m-%d")
    buy_date = datetime.strptime(buy_date, "%Y-%m-%d")
    investment_days = (sell_date - buy_date).days
    
    interest_earned = (investment_days * buy_price * guaranteed_rate) / 365
    
    # Calculate coupons received between buy and sell dates
    coupon_received_per_bond = sum(
        row["Cashflow"] for _, row in cashflows.iterrows()
        if buy_date < datetime.strptime(row["Date"], "%Y-%m-%d") <= sell_date
    )
    
    sell_price = buy_price + interest_earned - coupon_received_per_bond
    
    print(f"Investment Days: {investment_days}")
    print(f"Interest Earned: {interest_earned:.2f}")
    print(f"Total Coupon Received: {coupon_received_per_bond:.2f}")
    print(f"Final Sell Price: {sell_price:.2f}")
    
    return sell_price

# Example usage
face_value = 100000
coupon_rate = 0.105
issue_date = "2021-03-30"
maturity_date = "2024-03-30"
yield_rate = 0.11
guaranteed_rate = 0.11
buy_date = "2021-04-26"
sell_date = "2021-05-07"

cashflows = generate_cashflow(face_value, coupon_rate, issue_date, maturity_date)
buy_price = calculate_buy_price(cashflows, buy_date, coupon_rate, face_value)
sell_price = calculate_sell_price(buy_price, sell_date, buy_date, guaranteed_rate, cashflows)

print(f"Final Buy Price: {buy_price:.2f}")
print(f"Final Sell Price: {sell_price:.2f}")


Generated Cashflows:
         Date       Cashflow
0  2021-03-30   10500.000000
1  2022-03-30   10500.000000
2  2023-03-30   10500.000000
3  2024-03-30  110528.767123
Discounting 10500.0 on 2022-03-30 with factor 1.095030: 9588.78
Discounting 10500.0 on 2023-03-30 with factor 1.207818: 8693.36
Discounting 110528.76712328767 on 2024-03-30 with factor 1.332581: 82943.35
Total Present Value of Cashflows: 101225.49
Computed Buy Price: 101225.49
Investment Days: 11
Interest Earned: 335.57
Total Coupon Received: 0.00
Final Sell Price: 101561.06
Final Buy Price: 101225.49
Final Sell Price: 101561.06
